In [615]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [616]:
# !pip install gurobipy


In [617]:
from gurobipy import *

### Step 1: compute minimum tardy amount t

##### Constants
$\text{Let}$  <br>

$M = \{1, 2, ..., 5\} \text{ be the set of machines,}$<br>

$J = \{1, 2, ..., 12\} \text{ be the set of jobs,}$  <br>

$I = \{1, 2, ..., 6\}\text{ be the set of process number,}$ <br>

$S_{j} = \text{ the splitting timing for job } j \text{, } \forall j \in J \text{, }$ <br>

$D_{j} = \text{ the due time for job } j \text{, } \forall j \in J \text{, }$ <br>

$P = \{boiling, baking, smoking, null\} \text{ be the set of process name,}$ <br>

$ C_{mp} = 
\begin{cases}
    \ 1, & \text{if machine } m \text{ can do process } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall m \in M, p \in P
$

$A_{ijp} = 
\begin{cases}
    \ 1, & \text{if process } i \text{ in job } j \text{ is } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall i \in I, j \in J, p \in P\text{, }$

$T_{ij} = \text{time spent by process } i \text{ of job } j \text{ (} T_{ij} = 0 \text{ if } \sum_{p \in P}{A_{ijp}} = 0 \text{) } \forall i \in I, j \in J \text{. }$

$ b= \text{the opening time of the smoking station}$


##### Variables
$\text{def.}$


$ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$

$w_{j} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ is tardy job }\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I
$

<!-- $ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$ -->

$z_{i,j,i',j'} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ of process }i \text{ is scheduled before job }j' \text{ of process } i'\\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J
$

$ y_{i,j,i',j',m} =
\begin{cases}
    \ 1, & \text{if } x_{i,j,m} = x_{i',j',m} = 1 \\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J, m \in M
$

$ v_{ij} = \text{the completion time of process }i \text{, job }j, \forall i \in I, j \in J
$







$\text{obj. }$ $\min \sum_{j \in J} w_j$ 

$\text{s.t. }$  \
ensure the correctness of order of each process 

**1-a. same job** \
$v_{ij} + T_{ij} - v_{i'j} \leq K \cdot z_{iji'j} \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$ \
$v_{i'j} + T_{i'j} - v_{ij} \leq K \cdot (1-z_{iji'j}) \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$   

**1-b. same machine** \
$y_{iji'j'm}(v_{ij} \cdot x_{ijm} + T_{ij} - v_{i'j'} \cdot x_{i'j'm}) \leq K \cdot z_{iji'j'} \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$ \
$y_{iji'j'm}(v_{i'j'} \cdot x_{i'j'm} + T_{i'j'} - v_{ij} \cdot x_{ijm}) \leq K \cdot (1 - z_{iji'j'}) \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$

**2. Each process is completed by exactly one machine**

$$\sum_{m \in M}x_{ijm} = 1, \forall i \in I, j \in J$$

**3. Whether a machine can do to process type** \
$x_{ijm} \cdot A_{ijp} \leq C_{mp} \quad \forall p \in P, i \in I, j \in J, m \in M$

**4. Constraint related to "Split"**

$$v_{i+1j}-v_{ij}=T_{i+1j}, \forall i \in I - \{S_{j}, |I|\}, j \in J$$ :      連續

$$x_{i+1jm} = x_{ijm}, \forall i \in I - \{S_{j}, |I|\}, m \in M, j \in J$$:    同一台機器

**5. Minimum Tardiness**\
$v_{|I|j} - (D_j - b) \geq K \cdot w_j \quad \forall j \in J$ 

**6. Non-negative constraints** \
$w_{ij} \in \{0,1\}$


### Step 2: minimize makespan

$\text{obj. } \min \: (\max v_{ij})$ \
$\text{s.t.}$ \
$\sum_{j \in J}w_j = t$  
$\text{and all above constraints.}$ 

In [618]:
import pandas as pd

In [619]:
# data = pd.read_excel('openpyxl","data/OR110-1_case01.xlsx', engine='openpyxl')
df1 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 1')
df2 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 2')
df3 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 3')



In [620]:
df1

,Job ID,Process 1,Process 2,Splitting Timing,Process 1.1,Process 2.1,Due Time
0,1,Boiling,Baking,1,2.7,1.3,12:30:00
1,2,Baking,Boiling,1,1.6,1.4,12:30:00
2,3,Boiling,Baking,1,0.7,1.9,12:30:00
3,4,Smoking,Boiling,1,0.5,0.7,12:30:00
4,5,Boiling,Baking,1,0.8,1.0,12:30:00
5,6,Boiling,NaN,0,2.5,NaN,12:30:00
6,7,Baking,Smoking,1,1.4,2.0,12:30:00
7,8,Baking,Smoking,1,1.1,1.1,17:30:00
8,9,Boiling,Baking,1,0.8,0.7,17:30:00
9,10,Baking,Boiling,1,1.0,0.5,17:30:00


In [621]:
maxProcess = 2
I = []
totalLength = len(df1)
for i in range(totalLength):
    temp = []
    maxI = 0
    for j in range(maxProcess):
        columnName = "Process " + str(j+1) + ".1"
        print("Column Name = ", columnName, ", item index = ", i, ", value = ", df1[columnName][i])
        if(np.isnan(df1[columnName][i])):
            break
        else:
            maxI += 1
    I.append(maxI)
I

Column Name =  Process 1.1 , item index =  0 , value =  2.7
Column Name =  Process 2.1 , item index =  0 , value =  1.3
Column Name =  Process 1.1 , item index =  1 , value =  1.6
Column Name =  Process 2.1 , item index =  1 , value =  1.4
Column Name =  Process 1.1 , item index =  2 , value =  0.7
Column Name =  Process 2.1 , item index =  2 , value =  1.9
Column Name =  Process 1.1 , item index =  3 , value =  0.5
Column Name =  Process 2.1 , item index =  3 , value =  0.7
Column Name =  Process 1.1 , item index =  4 , value =  0.8
Column Name =  Process 2.1 , item index =  4 , value =  1.0
Column Name =  Process 1.1 , item index =  5 , value =  2.5
Column Name =  Process 2.1 , item index =  5 , value =  nan
Column Name =  Process 1.1 , item index =  6 , value =  1.4
Column Name =  Process 2.1 , item index =  6 , value =  2.0
Column Name =  Process 1.1 , item index =  7 , value =  1.1
Column Name =  Process 2.1 , item index =  7 , value =  1.1
Column Name =  Process 1.1 , item index 

[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1]

# Problem 1

In [622]:
import numpy as np

In [623]:
K = 1000000

In [624]:
from datetime import datetime, date

date_time_str = "7:30:00"

b = datetime.strptime(date_time_str, '%H:%M:%S').time()


print ("The date is", b)

The date is 07:30:00


In [625]:
S = []
D = []
for index, row in df1.iterrows():
    S.append(row['Splitting Timing'])
    D.append(row['Due Time'])
D

[datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30)]

In [626]:
for i in range(len(S)):
    S[i] = 0
S

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [627]:
peg1 = Model("peg1")

M = 4
# I = 2
J = 12
method = ["Boiling", "Baking", "Smoking"]
# method.append("null")
P = len(method)

A = {}
for j in range(J):
    # A.append([])
    for i in range(I[j]):
        # A[i].append([])
        count = 0
        for p in range(P-1):
            if(df1["Process "+ str(i+1)][j] == method[p]):
                A[i,j,p] = 1
                count = 1
            else:
                A[i,j,p] = 0
        if(count == 0):
            A[i, j, P-1] = 1
        else:
            A[i, j, P-1] = 0
A

{(0, 0, 0): 1,
 (0, 0, 1): 0,
 (0, 0, 2): 0,
 (1, 0, 0): 0,
 (1, 0, 1): 1,
 (1, 0, 2): 0,
 (0, 1, 0): 0,
 (0, 1, 1): 1,
 (0, 1, 2): 0,
 (1, 1, 0): 1,
 (1, 1, 1): 0,
 (1, 1, 2): 0,
 (0, 2, 0): 1,
 (0, 2, 1): 0,
 (0, 2, 2): 0,
 (1, 2, 0): 0,
 (1, 2, 1): 1,
 (1, 2, 2): 0,
 (0, 3, 0): 0,
 (0, 3, 1): 0,
 (0, 3, 2): 1,
 (1, 3, 0): 1,
 (1, 3, 1): 0,
 (1, 3, 2): 0,
 (0, 4, 0): 1,
 (0, 4, 1): 0,
 (0, 4, 2): 0,
 (1, 4, 0): 0,
 (1, 4, 1): 1,
 (1, 4, 2): 0,
 (0, 5, 0): 1,
 (0, 5, 1): 0,
 (0, 5, 2): 0,
 (0, 6, 0): 0,
 (0, 6, 1): 1,
 (0, 6, 2): 0,
 (1, 6, 0): 0,
 (1, 6, 1): 0,
 (1, 6, 2): 1,
 (0, 7, 0): 0,
 (0, 7, 1): 1,
 (0, 7, 2): 0,
 (1, 7, 0): 0,
 (1, 7, 1): 0,
 (1, 7, 2): 1,
 (0, 8, 0): 1,
 (0, 8, 1): 0,
 (0, 8, 2): 0,
 (1, 8, 0): 0,
 (1, 8, 1): 1,
 (1, 8, 2): 0,
 (0, 9, 0): 0,
 (0, 9, 1): 1,
 (0, 9, 2): 0,
 (1, 9, 0): 1,
 (1, 9, 1): 0,
 (1, 9, 2): 0,
 (0, 10, 0): 1,
 (0, 10, 1): 0,
 (0, 10, 2): 0,
 (1, 10, 0): 0,
 (1, 10, 1): 1,
 (1, 10, 2): 0,
 (0, 11, 0): 0,
 (0, 11, 1): 0,
 (0, 11, 2): 1}

In [628]:
T = {}
for j in range(J):
    # temp = []
    for i in range(I[j]):
        if(np.isnan(df1["Process " + str(i+1) + ".1"][j])):
            # temp.append(0)
            T[i, j] = 0
        else:
            # temp.append(j)
            T[i, j] = df1["Process " + str(i+1) + ".1"][j]
    # T.append(temp)
T

{(0, 0): 2.7,
 (1, 0): 1.3,
 (0, 1): 1.6,
 (1, 1): 1.4,
 (0, 2): 0.7,
 (1, 2): 1.9,
 (0, 3): 0.5,
 (1, 3): 0.7,
 (0, 4): 0.8,
 (1, 4): 1.0,
 (0, 5): 2.5,
 (0, 6): 1.4,
 (1, 6): 2.0,
 (0, 7): 1.1,
 (1, 7): 1.1,
 (0, 8): 0.8,
 (1, 8): 0.7,
 (0, 9): 1.0,
 (1, 9): 0.5,
 (0, 10): 3.0,
 (1, 10): 1.4,
 (0, 11): 2.0}

In [629]:

C = {}
machineNumber = 4
for i in range(machineNumber):
    for j in range(P):
        C[i, j] = 1
#     C[i, P-1] = 0
C


{(0, 0): 1,
 (0, 1): 1,
 (0, 2): 1,
 (1, 0): 1,
 (1, 1): 1,
 (1, 2): 1,
 (2, 0): 1,
 (2, 1): 1,
 (2, 2): 1,
 (3, 0): 1,
 (3, 1): 1,
 (3, 2): 1}

In [630]:
### variables
# add x_ijm
x = {}
# x = []
for j in range(J):
    # x.append([])
    for i in range(I[j]):
        # x[i].append([])
        for m in range(M):
            # x[i][j].append(peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2)));
             x[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2))

# add w_j
w = {}
for j in range(J):
    w[j] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "w"+str(j+1))

# add z_iji'j'
z = {}
for j in range(J):
    # z.append([])
    for i in range(I[j]):
        # z[i].append([])
        for j_ in range(J):
            # z[i][j].append([])
            for i_ in range(I[j_]):
                z[i,j,i_,j_] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1))

# add y_iji'j'm
y = {}
for j in range(J):
    # y.append([])
    for i in range(I[j]):
        # y[i].append([])
        for j_ in range(J):
            # y[i][j].append([])
            for i_ in range(I[j_]):
                # y[i][j][i_].append([])
                for m in range(M):
                    y[i,j,i_,j_,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1) + str(m+2))

# add v_ij
v = {}
for j in range(J):
    # v.append([])
    for i in range(I[j]):
        v[i, j] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "v" + str(i+1) + str(j+1))


In [631]:
### obj
peg1.setObjective(quicksum(w[j] for j in range(J)), GRB.MINIMIZE)
# 1
peg1.addConstrs((((((v[i, j] + T[i,j] - v[i_, j] <= K * z[i, j,i_,j]) for j in range(J-1)) for i in range(I[j]-1)) for j_ in range(j+1, J)) for i_ in range(i+1, I[j_])), "same_job1")

peg1.addConstrs((((((v[i_,j] + T[i,j] - v[i,j] <= K * z[i,j,i_,j]) for j in range(J-1)) for i in range(I[j]-1)) for j_ in range(j+1, J)) for i_ in range(i+1, I[j_])),"same_job2")



{}

In [632]:
# add vx_ijm
vx = {}
for j in range(J):
    for i in range(I[j]):
        for m in range(M):
            vx[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "vx" + str(i+1) + str(j+1)+str(m+2))



In [633]:
# peg1.addConstrs((vx[i,j,m] == v[i,j] * x[i,j,m] for j in range(J) for i in range(I[j]) for m in range(M)), "vx")
peg1.addConstrs((y[i,j,i_,j_,m] == x[i,j,m]*x[i_,j_,m] 
                for j in range(J-1)
                for i in range(I[j]-1) 
                for j_ in range(j+1,J)
                for i_ in range(i+1,I[j_]) 
                for m in range(M)),
                "y_constraint")

{(0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 3): <gurobi.QConstr Not Yet Added>,


In [634]:
# 1-a
peg1.addConstrs(((v[i,j] + T[i_,j] - v[i_,j]) <= K * (1 - z[i,j,i_,j])
    for j in range(J)
    for i in range(I[j]-1) 
    for i_ in range(i+1, I[j])),"same_job1")

# 1-a
peg1.addConstrs(((v[i_,j] + T[i,j] - v[i,j]) <= K * z[i,j,i_,j]
    for j in range(J)
    for i in range(I[j]-1) 
    for i_ in range(i+1, I[j])),"samejob2")

{(0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (10, 0, 1): <gurobi.Constr *Awaiting Model Update*>}

In [635]:
# 1-b
peg1.addConstrs(((y[i,j,i_,j_,m] * ((v[i,j] + T[i_,j_]) - v[i_,j_])) <= (K * (1 - z[i,j,i_,j_]) )
                for j in range(J-1)  
                for j_ in range(j+1, J)
                 for i in range(I[j]-1)  
                for i_ in range(i+1, I[j_])  
                for m in range(M)),"same_machine1")

# 1-b
peg1.addConstrs((y[i,j,i_,j_,m] * (v[i_,j_] + T[i,j] - v[i,j]) <= K * z[i,j,i_,j_]
    for j in range(J-1) 
    for i in range(I[j]-1)
    for j_ in range(j+1, J)
    for i_ in range(i+1, I[j_])
    for m in range(M)),"same_machine2")

{(0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 2, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 3, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 4, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 6, 1, 3): <gurobi.QConstr Not Yet Added>,


In [636]:
import math

In [637]:
# peg1.addConstrs(((x[i,j,m] == 0) >> y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint1')

# peg1.addConstrs(((x[i,j_,m] == 0) >>  y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint2')


In [638]:
# 2
peg1.addConstrs((quicksum(x[i,j,m] for m in range(M)) == 1 for j in range(J) for i in range(I[j]) ), "each process is completed by exactly one machine")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 1): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>,
 (9, 1): <gurobi.Constr *Awaiting Model Update*>,
 (10, 0): <gurobi.Constr *Awaiting Model Update*>,

In [639]:
# 3
peg1.addConstrs((x[i,j,m] * A[i,j,p] <= C[m,p] for j in range(J) for i in range(I[j]) for p in range(P) for m in range(M)), "whether a machine can do to process type")


{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 1): <gurobi.Constr *Awaiting Model Up

In [640]:
J

12

In [641]:
# 4
peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for j in range(J) for i in range(S[j]-1)), "split1-1")

peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for j in range(J) for i in range(S[j], I[j]-1)), "split1-2")

peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for j in range(J) for i in range(S[j]-1) for m in range(M)), "split2-1")

peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for j in range(J) for i in range(S[j], I[j]-1) for m in range(M)), "split2-2")    


{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0, 2): <gurobi.Constr *Awaiting Model Upd

In [642]:
# (datetime.combine(date.today(), D[0]) - datetime.combine(date.today(), b)).total_seconds()

In [643]:
# 5
peg1.addConstrs(
    (v[I[j]-1,j] - ((datetime.combine(date.today(), D[j]) - datetime.combine(date.today(), b)).total_seconds()/3600) <= K*w[j] 
    for j in range(J)),
    "minimum tardiness")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>}

In [644]:
peg1.addConstrs((v[i+1,j] - v[i,j] >= 0 for j in range(J) for i in range(I[j]-1)), "process order of same job")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>,
 (10, 0): <gurobi.Constr *Awaiting Model Update*>}

In [645]:
peg1.addConstrs((v[0,j]  >= T[0,j] for j in range(J)), "first process completion time")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>}

In [646]:
peg1.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 390 rows, 2630 columns and 392 nonzeros
Model fingerprint: 0x39e290a1
Model has 540 quadratic constraints
Variable types: 110 continuous, 2520 integer (2520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+06]
  QRHS range       [1e+06, 1e+06]
Presolve removed 362 rows and 2337 columns
Presolve time: 0.01s
Presolved: 2188 rows, 833 columns, 5656 nonzeros
Variable types: 378 continuous, 455 integer (455 binary)
Found heuristic solution: objective 6.0000000

Root relaxation: objective 0.000000e+00, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

In [647]:
for var in peg1.getVars():
  print(var.varName, " = ", var.x)  
print("objective value =", peg1.objVal)
# # peg1.computeIIS()
# # peg1.write("model.ilp")

x112  =  0.0
x113  =  1.0
x114  =  0.0
x115  =  0.0
x212  =  0.0
x213  =  1.0
x214  =  0.0
x215  =  0.0
x122  =  0.0
x123  =  0.0
x124  =  0.0
x125  =  1.0
x222  =  0.0
x223  =  0.0
x224  =  0.0
x225  =  1.0
x132  =  0.0
x133  =  1.0
x134  =  0.0
x135  =  0.0
x232  =  0.0
x233  =  1.0
x234  =  0.0
x235  =  0.0
x142  =  0.0
x143  =  0.0
x144  =  1.0
x145  =  0.0
x242  =  0.0
x243  =  0.0
x244  =  1.0
x245  =  0.0
x152  =  0.0
x153  =  0.0
x154  =  0.0
x155  =  1.0
x252  =  0.0
x253  =  0.0
x254  =  0.0
x255  =  1.0
x162  =  1.0
x163  =  0.0
x164  =  0.0
x165  =  0.0
x172  =  1.0
x173  =  0.0
x174  =  0.0
x175  =  0.0
x272  =  1.0
x273  =  0.0
x274  =  0.0
x275  =  0.0
x182  =  1.0
x183  =  0.0
x184  =  0.0
x185  =  0.0
x282  =  1.0
x283  =  0.0
x284  =  0.0
x285  =  0.0
x192  =  0.0
x193  =  0.0
x194  =  0.0
x195  =  1.0
x292  =  0.0
x293  =  0.0
x294  =  0.0
x295  =  1.0
x1102  =  0.0
x1103  =  0.0
x1104  =  1.0
x1105  =  0.0
x2102  =  0.0
x2103  =  0.0
x2104  =  1.0
x2105  =  0.0
x111

In [648]:
# peg1.computeIIS()
# peg1.write("model.ilp")

# Problem 2

# Problem 3

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d46e1ceb-56b3-4729-a7f8-5832e58e15c6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>